In [1]:
# personal imports
import prepare
import explore

# typical imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# modeling methods
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

# working with dates
from datetime import datetime
import time
from datetime import timedelta

# to evaluated performance using rmse
from sklearn.metrics import mean_squared_error
from math import sqrt 

# for tsa 
import statsmodels.api as sm

# holt's linear trend model. 
from statsmodels.tsa.api import Holt

import re

#clean look
import warnings
warnings.filterwarnings("ignore")

prep_tiktok(df)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4734 entries, 2022-09-16 00:00:00+00:00 to 2022-09-16 00:00:00+00:00
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   views    4734 non-null   int64 
 1   hashtag  4734 non-null   object
dtypes: int64(1), object(1)
memory usage: 111.0+ KB


/Users/jgardin875/codeup-data-science/Brad-file/prepare.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.views = df.views.astype('int')


<Figure size 432x288 with 0 Axes>

In [2]:
l = pd.read_csv('yt_meme_update.csv')

In [3]:
l.head()

,title,publishedTime,Unnamed: 2,date_ref,pub_date,duration,viewCount,hashtag
0,The Truth Behind The Global Pandemic. Jordan M...,18,0.0,9/16/22,9/16/22,26:40:00,48882,comedy
1,The Best Comedy Special Ever. Gabriel Rutledge...,1,30.5,9/16/22,8/16/22,25:24:00,621694,comedy
2,Look At These Assholes: Trump Family Edition |...,2,30.5,9/16/22,7/17/22,5:45,2248400,comedy
3,More Than Funny Comedy Special | Michael Jr. #...,11,30.5,9/16/22,10/15/21,1:23:38,1813052,comedy
4,2020 FUNNY COMEDY BEST OF MARVELOUS SARAH PRIN...,1,365.0,9/16/22,9/16/21,28:51:00,21036895,comedy


In [4]:
m = l.duration.str.split(r':', expand=True)

In [ ]:
#self check. The first m_s item is 26 min and 40 sec

In [37]:
m.head()

,hours,mins,sec
0,26,40,0
1,25,24,0
2,5,45,0
3,1,23,38
4,28,51,0


In [6]:
m.rename(columns = {0:'hours', 1:'mins', 2:'sec'}, inplace=True)

In [7]:
m.sec.fillna(0, inplace=True)

In [8]:
m.mins.fillna(0, inplace=True)

In [9]:
m.sec = m.sec.astype('int')

In [10]:
m.sort_values('sec')

,hours,mins,sec
0,26,40,0
3088,0,11,0
3087,0,09,0
3086,0,16,0
3085,0,24,0
...,...,...,...
4518,1,08,43
3691,2,01,47
87,1,02,49
1402,1,01,55


In [11]:
m_s = m[m.sec == 0]

In [12]:
m_s.shape

(4600, 3)

In [13]:
m_s.head()

,hours,mins,sec
0,26,40,0
1,25,24,0
2,5,45,0
4,28,51,0
6,8,39,0


In [14]:
zeros = m_s.sec

In [15]:
zeros = pd.DataFrame(zeros)

In [16]:
zeros.rename(columns = {'sec':"Z"}, inplace=True)

In [17]:
attempt1 = pd.concat([zeros, m_s], axis = 1)

In [18]:
attempt1.head()

,Z,hours,mins,sec
0,0,26,40,0
1,0,25,24,0
2,0,5,45,0
4,0,28,51,0
6,0,8,39,0


In [19]:
attempt1.rename(columns= {'Z':'A', 'hours':'B', 'mins':'C', 'sec':'D' }, inplace = True)

In [20]:
attempt1.drop(columns = ['D'], inplace=True)

In [21]:
attempt1.rename(columns = {'A':'hours', 'B':'mins', 'C':'sec'}, inplace = True)

In [22]:
attempt1.head()

,hours,mins,sec
0,0,26,40
1,0,25,24
2,0,5,45
4,0,28,51
6,0,8,39


In [23]:
m_s = attempt1

In [24]:
h_m_s = m[m.sec !=0]

In [25]:
h_m_s.head()

,hours,mins,sec
3,1,23,38
5,1,06,35
7,1,54,22
14,1,33,27
87,1,02,49


In [38]:
h_m_s.tail()

,hours,mins,sec
4249,6,33,23
4254,1,00,14
4280,2,02,10
4333,15,00,4
4518,1,08,43


In [ ]:
# self check. The last h_m_s item is 1 hr. 8 min and 43 sec

In [26]:
m_s.head()

,hours,mins,sec
0,0,26,40
1,0,25,24
2,0,5,45
4,0,28,51
6,0,8,39


In [27]:
h_m_s.head()

,hours,mins,sec
3,1,23,38
5,1,06,35
7,1,54,22
14,1,33,27
87,1,02,49


In [28]:
b_time = pd.concat([m_s, h_m_s], ignore_index=True)

In [29]:
b_time.head()

,hours,mins,sec
0,0,26,40
1,0,25,24
2,0,5,45
3,0,28,51
4,0,8,39


In [30]:
b_time.hours = b_time.hours.astype('int');

In [31]:
b_time.mins= b_time.mins.astype('int');

In [32]:
b_time.sec= b_time.sec.astype('int');

In [33]:
b_time['tot_sec'] = (b_time.hours*3600)+(b_time.mins*60)+(b_time.sec)

In [ ]:
#self check. the first m_s item is 26 min adn 40 sec 

In [34]:
b_time.head()

,hours,mins,sec,tot_sec
0,0,26,40,1600
1,0,25,24,1524
2,0,5,45,345
3,0,28,51,1731
4,0,8,39,519


In [ ]:
# self check the last h_m_s item is 1 hr 8 min and 43 sec

In [35]:
b_time.tail()

,hours,mins,sec,tot_sec
4629,6,33,23,23603
4630,1,0,14,3614
4631,2,2,10,7330
4632,15,0,4,54004
4633,1,8,43,4123
